# Notebook para la extracción de Tweets y características mediante el análisis de Tweets clasificados como odio

In [ ]:
!pip install tokenizers==0.10.3
!pip install transformers==4.16.2
!pip install tweepy==4.8.0
!pip install nltk==3.4
!pip install flair
!pip install subject-classification-spanish
!pip install pyspellchecker
!pip install pysentimiento

     |████████████████████████████████| 3.3 MB 7.9 MB/s 
     |████████████████████████████████| 3.5 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 54.2 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 880 kB 46.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=913b5de16389d195348bc0cb9be8a3c3ddcc452a023a30e033724f23d5d1a4dd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 77 kB 4.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-

In [ ]:
import tweepy
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import time
from flair.models import SequenceTagger
from flair.data import Sentence
from bs4 import BeautifulSoup
import re
import os
from transformers import pipeline
from urllib.parse import urlparse
from spellchecker import SpellChecker
from subject_classification_spanish import subject_classifier
import glob
import operator
import itertools
import numpy as np
import collections
from urllib.request import urlopen
from nltk.tag.stanford import StanfordPOSTagger
from nltk.util import ngrams
from google.colab import drive
import random

drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/data"
data_path = "/content/gdrive/My Drive/data/"

APIKey = "hNBmOgS4wIpquzX3oz6pyjm9O"
APISecret = "JbjktlvxAM13A4UQO6ac89qNzFQEQtUITRP5ftJxkkDuFhsUrb"

BearerToken = "AAAAAAAAAAAAAAAAAAAAAP8AVAEAAAAAHrVncHzSWORJyuOA30JGvRGY9ts%3DaMxPz4jfk58AdPcjfgmZDxwnLKEk5E9jHGmGJaUFyc6EmLIfcS"

AccessToken = "1170021624-Y5QVCqMy2KiHP09eFq9dm58fwBWEc4GMiFpfZBU"
AccessSecret = "tlW1MYq62YQa9myZ7XPz8hBvagDZhTuNhcwSXcNF8eFWf"

Mounted at /content/gdrive
beto-sentiment-analysis     datasets	  seed_periodicos.txt
caracteristicas_gloria.txt  keywords	  stanford-postagger-full-2020-11-17
contenido_periodicos	    lexicon	  tensorflow
csvs_experimentos_NB1	    models_saved


In [ ]:
def connect_to_api():
    auth = tweepy.OAuthHandler(APIKey, APISecret)
    auth.set_access_token(AccessToken, AccessSecret)

    api = tweepy.API(auth, wait_on_rate_limit=True)

    # probamos las claves
    try:
        api.verify_credentials()
        print("Authentication OK")
    except:
        print("Error during authentication")

    return api

def get_tweets(api, limit, geo, query):
    # tweets recientes publicados en los ultimos 7 dias
    tweets = [status for status in tqdm(tweepy.Cursor(api.search_tweets, q=query, geocode=geo).items(limit), unit="tw", total=limit)]

    return tweets 

leet_alph = {'0':'o', '1':'i', '3':'e', '5':'s', '4':'a', '7':'t', '8': 'b'}

regex = "/*[a-zA-ZñÑ$]*[0134578$][a-zA-ZñÑ]*"
regex2 = "https://t.co/\w*"
regex3 = "@[a-zA-Z0-9]*"
regex4 = "#[a-zA-Z0-9]*"
spell = SpellChecker(language='es')

### Este es de huggingface
path_ = "finiteautomata/beto-sentiment-analysis"
classify_ = pipeline("sentiment-analysis", model=path_, tokenizer=path_)

### Este modelo es mio preentrenado
path2_ = data_path + "models_saved/loocv_bert-base-spanish-wwm-cased_Spanish_translated_baseline_100"
classify2_ = pipeline("sentiment-analysis", model=path2_, tokenizer=path2_)

cats_classifier = subject_classifier.SubjectClassifier()

regex_bad_words_ = re.compile("(" + "|".join(pd.concat([pd.read_csv(f) for f in glob.glob(data_path + 'lexicon/*.txt')], ignore_index=True)["termino"].values) + ")")

def leet_converter(word):
    for k, v in leet_alph.items():
        word = word.replace(k,v)
    return word

class Atributos:
    def __init__(self):
        self.end_date = 0
        self.start_date = 0
        self.geo_enabled_tweets = 0
        self.retweets = 0
        self.negativos, self.positivos, self.neutros = 0, 0, 0
        self.hate, self.no_hate = 0, 0
        self.baddies = []
        self.n_baddies = 0
        self.len_status = 0
        self.leet_counter = 0
        self.misspelling_counter = 0
        self.neg_score, self.pos_score, self.neu_score = 0, 0, 0
        self.times_user_quotes= 0
        self.times_user_rt = 0
        self.num_rts_to_tweets = 0
        self.num_favs_to_tweets = 0
        self.no_hate_score = 0
        self.hate_score = 0
        self.neg_score = 0
        self.pos_score = 0
        self.neu_score = 0
        self.no_hate_score = 0
        self.hate_score = 0

        self.detected_hashtags = collections.Counter()
        self.retweeted_users = collections.Counter()
        self.mentioned_users = collections.Counter()
        self.detected_domains = collections.Counter()

        self.user_categories = {}
        self.id_screen_names = {}


def process_tweet(tweet, attrs):
    tw_date = tweet.created_at

    # Updating most recent tweet
    attrs.end_date = attrs.end_date or tw_date
    attrs.start_date = tw_date

    # Sentiment / hate
    if tweet.text:
        entrada = str(tweet.text)
        entrada = entrada.split(" ")
        new_entrada = []

        for w in entrada:
            if len(w) > 3:
                url = re.findall(regex2, w)
                us_mention = re.findall(regex3, w)
                us_hashtag = re.findall(regex4, w)
                if url or us_mention or us_hashtag:
                    w = ""
                leetword = re.findall(regex, w)
                if leetword:
                    w = leet_converter(leetword[0])
                    attrs.leet_counter += 1
            new_entrada.append(w)
        
        salida = []
        misspelled = spell.unknown(new_entrada)
        for word in new_entrada:
            re.sub(r"[^\w\s]", "", str(word))
            for m in misspelled:
                if len(word) > 3 and word == m:
                    #print(m)
                    word = spell.correction(m)
                    attrs.misspelling_counter += 1
            salida.append(word)

        entrada = " ".join(salida)

        result = classify_(str(entrada))
        result2 = classify2_(str(entrada))

        if result[0]['label'] == 'NEG':
            attrs.negativos += 1
            attrs.neg_score += float(result[0]['score'])
        elif result[0]['label'] == 'POS':
            attrs.positivos += 1
            attrs.pos_score += float(result[0]['score'])
        elif result[0]['label'] == 'NEU':
            attrs.neutros += 1
            attrs.neu_score += float(result[0]['score'])
        if result2[0]['label'] == 'LABEL_0':
            attrs.no_hate += 1
            attrs.no_hate_score += float(result[0]['score'])
        elif result2[0]['label'] == 'LABEL_1':
            attrs.hate += 1
            attrs.hate_score += float(result[0]['score'])
        
        # More textual data
        attrs.baddies += regex_bad_words_.findall(str(entrada))
        attrs.n_baddies = len(attrs.baddies)  
        attrs.len_status += len(str(entrada))
        attrs.num_rts_to_tweets += tweet.retweet_count
        attrs.num_favs_to_tweets += tweet.favorite_count

        classes_result = cats_classifier.classify(str(entrada))
        attrs.user_categories.update(dict(itertools.islice(classes_result.items(), 3)))
    
    if tweet.is_quote_status:
        attrs.times_user_quotes += 1

    if "retweeted_status" in tweet._json:
        attrs.times_user_rt += 1    

    # Detecting geolocation
    if tweet.place:
        attrs.geo_enabled_tweets += 1
    
    # Updating hashtags list
    if tweet.entities['hashtags']:
        for ht in tweet.entities['hashtags']:
            ht['text'] = "#%s" % ht['text']
            attrs.detected_hashtags[ht['text']] += 1
    
    # Handling retweets
    try:
        # We use id to get unique accounts (screen_name can be changed)
        rt_id_user = tweet.retweeted_status.user.id_str
        attrs.retweeted_users[rt_id_user] += 1

        if tweet.retweeted_status.user.screen_name not in attrs.id_screen_names:
            attrs.id_screen_names[rt_id_user] = "@%s" % tweet.retweeted_status.user.screen_name
        attrs.retweets += 1
    except:
        pass

    # Updating domains list
    if tweet.entities['urls']:
        for url in tweet.entities['urls']:
            domain = urlparse(url['expanded_url']).netloc
            if domain != "twitter.com":  # removing twitter.com from domains (not very relevant)
                attrs.detected_domains[domain] += 1

    # Updating mentioned users list
    if tweet.entities['user_mentions']:
        for ht in tweet.entities['user_mentions']:
            attrs.mentioned_users[ht['id_str']] += 1
            if not ht['screen_name'] in attrs.id_screen_names:
                attrs.id_screen_names[ht['id_str']] = "@%s" % ht['screen_name'] 

def obtener_caracteristicas(api, tweet, user, limit):
    diccionario_caracteristicas = {} 
    
    # atributos tabla C.3
    diccionario_caracteristicas['user_id'] = user.id
    diccionario_caracteristicas['user_name'] = user.name
    diccionario_caracteristicas['screen_name'] = user.screen_name
    diccionario_caracteristicas['descripcion'] = user.description
    diccionario_caracteristicas['localizacion'] = user.location
    
    diccionario_caracteristicas['verificado'] = user.verified
    diccionario_caracteristicas['profile_image_url'] = user.profile_image_url
    diccionario_caracteristicas['profile_changed'] = user.default_profile
    diccionario_caracteristicas['img_prof_changed'] = user.default_profile_image
    diccionario_caracteristicas['user_geo_enabled'] = user.geo_enabled
    diccionario_caracteristicas['fecha_creacion'] = user.created_at
    diccionario_caracteristicas['statuses_count'] = user.statuses_count
    diccionario_caracteristicas['listed_count'] = user.listed_count
    diccionario_caracteristicas['followers_count'] = user.followers_count
    diccionario_caracteristicas['followees_count'] = user.friends_count
    diccionario_caracteristicas['favourites_count'] = user.favourites_count

    # atributos tabla C.4
    diccionario_caracteristicas['text'] = tweet.text
    diccionario_caracteristicas['tweet_id'] = tweet.id
    diccionario_caracteristicas['tweet_creation_at'] = tweet.created_at
    diccionario_caracteristicas['n_favs'] = tweet.favorite_count
    diccionario_caracteristicas['n_rts'] = tweet.retweet_count

    rt, qt, rp ='','',''

    if "retweeted_status" in tweet._json and tweet._json["retweeted_status"] is not None:
        rt = tweet._json["retweeted_status"]
    if "quoted_status" in tweet._json and tweet._json["quoted_status"] is not None:
        qt = tweet._json["quoted_status"]
    if "in_reply_to_screen_name" in tweet._json and tweet._json["in_reply_to_screen_name"] is not None:
        rp = tweet._json["in_reply_to_screen_name"]
    

    # si el tweet es rt
    if rt:
        diccionario_caracteristicas['is_rt'] = True 
        diccionario_caracteristicas['rt_id_user'] = tweet.retweeted_status.user.id
        diccionario_caracteristicas['rt_id_status'] = tweet.retweeted_status.id
        diccionario_caracteristicas['rt_text'] = tweet.retweeted_status.text
        diccionario_caracteristicas['rt_creation_at'] = tweet.retweeted_status.created_at.timestamp()
        diccionario_caracteristicas['rt_fav_count'] = tweet.retweeted_status.favorite_count
        diccionario_caracteristicas['rt_rt_count'] = tweet.retweeted_status.retweet_count
    else:
        diccionario_caracteristicas['is_rt'] = False
        diccionario_caracteristicas['rt_id_user'] = 0
        diccionario_caracteristicas['rt_id_status'] = 0
        diccionario_caracteristicas['rt_text'] = ""
        diccionario_caracteristicas['rt_creation_at'] = ""
        diccionario_caracteristicas['rt_fav_count'] = 0
        diccionario_caracteristicas['rt_rt_count'] = 0

    # si el tweet es respuesta
    if rp: 
        diccionario_caracteristicas['is_reply'] = True
        diccionario_caracteristicas['reply_id_status'] = tweet.in_reply_to_status_id
        diccionario_caracteristicas['reply_id_user'] = tweet.in_reply_to_user_id
      
    else:
        diccionario_caracteristicas['is_reply'] = False
        diccionario_caracteristicas['reply_id_status'] = 0
        diccionario_caracteristicas['reply_id_user'] = 0

    # si el tweet es una cita de otro
    if qt and hasattr(tweet, 'quoted_status'):
        diccionario_caracteristicas['is_quote'] = True
        diccionario_caracteristicas['quote_id_status'] = tweet.quoted_status.id
        diccionario_caracteristicas['quote_id_user'] = tweet.quoted_status.user.id
        diccionario_caracteristicas['quote_text'] = tweet.quoted_status.text
        diccionario_caracteristicas['quote_creation_at'] = datetime.strptime(str(tweet.quoted_status.created_at)[:19], "%Y-%m-%d %H:%M:%S").timestamp()
        diccionario_caracteristicas['quote_fav_count'] = tweet.quoted_status.favorite_count
        diccionario_caracteristicas['quote_rt_count'] = tweet.quoted_status.retweet_count
    else:
        diccionario_caracteristicas['is_quote'] = False
        diccionario_caracteristicas['quote_id_status'] = 0
        diccionario_caracteristicas['quote_id_user'] = 0
        diccionario_caracteristicas['quote_text'] = ""
        diccionario_caracteristicas['quote_creation_at'] = ""
        diccionario_caracteristicas['quote_fav_count'] = 0
        diccionario_caracteristicas['quote_rt_count'] = 0

    # se obtiene con el clasificador VisualRecognitionV3, preguntar como usarlo
    # diccionario_caracteristicas['categories_profile_image_url'] = 
    # extraera todos los tweets del perfil a no ser que lleguen al limite de la API
    num_tweets = np.amin([limit, user.statuses_count])
    print("Tweets del usuario: {}".format(user.statuses_count))
    print("[+] Retrieving last {} tweets...".format(num_tweets))
    diccionario_caracteristicas['status_retrieving'] = num_tweets

    # para cada tweet descargado, lo procesamos y sacamos las fechas
    try:
        status = tweepy.Cursor(api.user_timeline, screen_name=user.screen_name).items(num_tweets)
    except Exception as e:
        print(e, 'User ->', user.name)   

    attrs = Atributos()
    for tweet in tqdm(status, unit="tw", total=limit):
        process_tweet(tweet, attrs)            
  
    if (attrs.end_date - attrs.start_date).days < 30 and (num_tweets < user.statuses_count):
         print("[!] Looks like we do not have enough tweets from user, you should consider changing limit value")
         diccionario_caracteristicas['status_note'] = "Looks like we do not have enough tweets from user, you should consider changing limit value"
    else:
        diccionario_caracteristicas['status_note'] = str("")

    if (attrs.end_date - attrs.start_date).days != 0:
        print("[+] Average number of tweets per day: %.1f" % (num_tweets / float((attrs.end_date - attrs.start_date).days)))
        diccionario_caracteristicas['status_average_tweets_per_day'] = (num_tweets / float((attrs.end_date - attrs.start_date).days))
    else:
        diccionario_caracteristicas['status_average_tweets_per_day'] = str("")

    diccionario_caracteristicas['geo_enabled_tweet_count'] = attrs.geo_enabled_tweets
    diccionario_caracteristicas['num_hashtags'] = len(attrs.detected_hashtags) 

    detected_hashtags1 = dict(sorted(attrs.detected_hashtags.items(), key=operator.itemgetter(1),reverse=True))
    diccionario_caracteristicas['top_hashtags'] = str(dict(itertools.islice(detected_hashtags1.items(), 10)))   
    

    diccionario_caracteristicas['rt_count'] = attrs.retweets
    # Converting users id to screen_names
    retweeted_users_names = {}
    for k in attrs.retweeted_users.keys():
        retweeted_users_names[attrs.id_screen_names[k]] = attrs.retweeted_users[k]

    retweeted_users_names1 = dict(sorted(retweeted_users_names.items(), key=operator.itemgetter(1),reverse=True))
    diccionario_caracteristicas['top_retweeted_users'] = str(dict(itertools.islice(retweeted_users_names1.items(), 5)))

    mentioned_users_names = {}
    for k in attrs.mentioned_users.keys():
        mentioned_users_names[attrs.id_screen_names[k]] = attrs.mentioned_users[k]

    mentioned_users_names1 = dict(sorted(mentioned_users_names.items(), key=operator.itemgetter(1),reverse=True))
    diccionario_caracteristicas['num_mentions'] = len(attrs.mentioned_users)
    diccionario_caracteristicas['top_mentioned_users'] = str(dict(itertools.islice(mentioned_users_names1.items(), 5)))  

    diccionario_caracteristicas['num_urls'] = len(attrs.detected_domains) 
    detected_domains1 = dict(sorted(attrs.detected_domains.items(), key=operator.itemgetter(1),reverse=True))
    diccionario_caracteristicas['top_referenced_domains'] = str(dict(itertools.islice(detected_domains1.items(), 6))) 

    diccionario_caracteristicas['negativos'] = attrs.negativos
    diccionario_caracteristicas['positivos'] = attrs.positivos
    diccionario_caracteristicas['neutros'] = attrs.neutros
    diccionario_caracteristicas['hate'] = attrs.hate
    diccionario_caracteristicas['no_hate'] = attrs.no_hate

    try:
        diccionario_caracteristicas['negativos_score'] = attrs.neg_score/attrs.negativos
    except ZeroDivisionError:
        diccionario_caracteristicas['negativos_score'] = 0
    try:
        diccionario_caracteristicas['positivos_score'] = attrs.pos_score/attrs.positivos
    except ZeroDivisionError:
        diccionario_caracteristicas['positivos_score'] = 0
    try:
        diccionario_caracteristicas['neutros_score'] = attrs.neu_score/attrs.neutros
    except ZeroDivisionError:
        diccionario_caracteristicas['neutros_score'] = 0
    try:
        diccionario_caracteristicas['hate_score'] = attrs.hate_score/attrs.hate
    except ZeroDivisionError:
        diccionario_caracteristicas['hate_score'] = 0
    try:
        diccionario_caracteristicas['no_hate_score'] = attrs.no_hate_score/attrs.no_hate
    except ZeroDivisionError:
        diccionario_caracteristicas['no_hate_score'] = 0
    try:
        hater_cond = attrs.hate/100*int(diccionario_caracteristicas['status_retrieving'])
    except ZeroDivisionError:
        hater_cond = 0

    if hater_cond >= 10 and attrs.negativos > attrs.positivos:
        diccionario_caracteristicas['is_hater'] = 1
    else:
        diccionario_caracteristicas['is_hater'] = 0    

    b = str(attrs.baddies)
    diccionario_caracteristicas['baddies'] = b
    diccionario_caracteristicas['n_baddies'] = attrs.n_baddies

    try:
        diccionario_caracteristicas['n_baddies_tweet'] = float(attrs.n_baddies/num_tweets)
    except ZeroDivisionError:
        diccionario_caracteristicas['n_baddies_tweet'] = 0  

    diccionario_caracteristicas['len_status'] = attrs.len_status/num_tweets    

    diccionario_caracteristicas['times_user_rt'] = attrs.times_user_rt
    diccionario_caracteristicas['times_user_quotes'] = attrs.times_user_quotes  

    diccionario_caracteristicas['num_rts_to_tweets'] = attrs.num_rts_to_tweets
    diccionario_caracteristicas['num_favs_to_tweets'] = attrs.num_favs_to_tweets

    diccionario_caracteristicas['top_categories'] = str(dict(itertools.islice(attrs.user_categories.items(), 15)))
    diccionario_caracteristicas['misspelling_counter'] = int(attrs.misspelling_counter)
    diccionario_caracteristicas['leet_counter'] = int(attrs.leet_counter)

    return diccionario_caracteristicas

def apply_pos_tagging_nltk(corpus):
    spanish_postagger = StanfordPOSTagger(data_path + '/stanford-postagger-full-2020-11-17/models/spanish-ud.tagger',
                                  data_path + '/stanford-postagger-full-2020-11-17/stanford-postagger.jar',
                                   encoding='utf8')
    
    fichero = open(data_path + "datasets/dataframe_pos_tagged.txt", "w+")                             
    for i, sentence in enumerate(corpus):
        print("Procesando el tweet nº {}".format(i+1))
        new_sentence = ""
        words = sentence.split()
        tagged_words = spanish_postagger.tag(words)
        for word, tag in tagged_words:
            if tag == 'VERB' or tag == 'ADJ':   
                new_sentence += word + " "
        if i == len(corpus) - 1:
          fichero.write(new_sentence[:-1])
        else:
          fichero.write(new_sentence[:-1] + "\n") 
    fichero.close()

def get_corpus(dataset):
    df = pd.read_csv(dataset)

    # obtengo el texto de las filas en las que el tweet
    # está etiquetado como odio
    corpus =  df.loc[df['label'] == 1, 'text'].tolist()
    
    # eliminamos enlaces 
    corpus = [re.sub(r'http\S+', '', word) for word in corpus]
    # eliminamos caracteres especiales y signos de puntuacion
    corpus_processed = [re.sub(r"[^a-zA-Z0-9_ÑñÁáÉéÍíÓóÚú]+", ' ', word) for word in corpus]

    return corpus_processed

def get_corpus_datasets_alreco(dataset):
    df = pd.read_csv(dataset)

    # obtengo el texto de las filas en las que el tweet
    # está etiquetado como odio
    corpus =  df.loc[df['majority'] == 'ofensivo-odio', 'Texto completo del Tweet'].tolist()
    
    # eliminamos enlaces 
    corpus = [re.sub(r'http\S+', '', word) for word in corpus]
    # eliminamos caracteres especiales y signos de puntuacion
    corpus_processed = [re.sub(r"[^a-zA-Z0-9_ÑñÁáÉéÍíÓóÚú]+", ' ', word) for word in corpus]

    return corpus_processed

def parse_keywords2(entidades_sin_duplicados):
    query = ""
    contador_entidades_query = 0
    list_queries = []
    # separar las entidades en palabras únicas
    # quedarnos solo con los apellidos?
    # que cada entidad no tenga espacios
    for i, entidad in enumerate(entidades_sin_duplicados):
        if contador_entidades_query == 20:
            list_queries.append(query[:-4])
            query = ""
            contador_entidades_query = 0
        else:
            query += entidad + " OR "
            contador_entidades_query += 1
            
    return list_queries

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MultiLabelBinarizer from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
# saco el dataset de donde aplicaremos bigrams y trigrams
# solo tweets que estén clasificados como odio
corpus = get_corpus_datasets_alreco(data_path + 'datasets/good_data_12_abril.csv')
# se podrian poner varios datasets, para que estuviera
# todo el rato buscando hasta que acabe el tiempo
apply_pos_tagging_nltk(corpus)

In [ ]:
with open (data_path + "datasets/dataframe_pos_tagged.txt", "r") as f:
  lines = f.read().splitlines()
f.close()

# bigrams
list_trigrams = []
for sentence in lines:
    trigrams = ngrams(sentence.split(), 3)
    for item in trigrams:
        list_trigrams.append(item)

print(list_trigrams)

[('vas', 'árabe', 'sientes'), ('árabe', 'sientes', 'cómodo'), ('sientes', 'cómodo', 'temo'), ('cómodo', 'temo', 'corta'), ('corrupto', 'podrido', 'aberrante'), ('podrido', 'aberrante', 'criminal'), ('aberrante', 'criminal', 'satánico'), ('criminal', 'satánico', 'Nuevo'), ('satánico', 'Nuevo', 'sois'), ('Nuevo', 'sois', 'grandísima'), ('sois', 'grandísima', 'poneros'), ('grandísima', 'poneros', 'vosotros'), ('poneros', 'vosotros', 'juntos'), ('vosotros', 'juntos', 'respetable'), ('vas', 'guapo', 'vas'), ('guapo', 'vas', 'enfrentar'), ('vas', 'enfrentar', 'preñada'), ('enfrentar', 'preñada', 'armados'), ('preñada', 'armados', 'sumisos'), ('armados', 'sumisos', 'sociocomunistas'), ('sumisos', 'sociocomunistas', 'últimos'), ('sociocomunistas', 'últimos', 'saber'), ('últimos', 'saber', 'tienen'), ('Tiene', 'nueva', 'llevaban'), ('nueva', 'llevaban', 'disfrazados'), ('cerrar', 'quiero', 'depende'), ('deas', 'trajo', 'perico'), ('CATALUÑA', 'detienen', 'radicalizados'), ('detienen', 'radicali

In [ ]:
actual_date = datetime.today()
csv_out = data_path + "/experimento_3.2_" + str(actual_date)
limite_tweets = 1
geo = geo = "40.416755,-3.703790,500km"
api = connect_to_api()

# tiempo en el que se quiere que se acabe (1h ahora mismo)
t_end = time.time() + 60 * 120
df_final = pd.DataFrame()
num_tweets_totales = 0
num_tweets_descartados = 0
start_time = time.time()

# cogemos una muestra aleatoria de trigrams 
trigrams_shuffled = random.sample(list_trigrams, k=len(list_trigrams))
list_queries_trigrams = []
query = ""
for (word1, word2, word3) in trigrams_shuffled:
  query = word1 + " AND " + word2 + " AND " + word3
  list_queries_trigrams.append(query)
  query = ""


try:
  while time.time() <= t_end:
      df_out = pd.DataFrame()
      
      # qué hacer con la lista de ngrams
      for query in list_queries_trigrams:
          print(query)
          tweets = get_tweets(api, limite_tweets, geo, query)
          
          for index, tweet in enumerate(tweets): 
              try:
                  tweet = api.get_status(tweet.id)
                  num_tweets_totales += 1
                  print("Obteniendo características del Tweet nº {}".format(num_tweets_totales))
                  caracteristicas = obtener_caracteristicas(api, tweet, tweet.user, 200)
                  df = pd.DataFrame([caracteristicas], columns=caracteristicas.keys())
                  df_out = pd.concat([df_out, df], axis=0)
              except tweepy.errors.TweepyException as e:
                  print("Tweepy Error: {}".format(e))
                  num_tweets_descartados += 1
                  continue
      df_final = pd.concat([df_final, df_out], axis=0)

except KeyboardInterrupt:
  df_out = pd.concat([df_out, df], axis=0)
  df_final = pd.concat([df_final, df_out], axis=0)

except Exception as e1:
  df_out = pd.concat([df_out, df], axis=0)
  df_final = pd.concat([df_final, df_out], axis=0)

end_time = time.time()
total_time = (end_time - start_time) / 60
print("{} minutos para obtener todas las caracteristicas de {} tweets".format(total_time, num_tweets_totales))
print("No se han podido obtener las características de {} Tweets".format(num_tweets_descartados))
df_final.to_csv(csv_out + "_sin_centralidad.csv", index = False)

Authentication OK
españoles AND estúpido AND cochina


  0%|          | 0/1 [00:00<?, ?tw/s]


colaboracionista AND quedaba AND largo


  0%|          | 0/1 [00:00<?, ?tw/s]


den AND entren AND veremos


...

  0%|          | 0/1 [00:00<?, ?tw/s]


inmediata AND nacional AND inmediata


100%|██████████| 1/1 [00:00<00:00,  5.22tw/s]


Obteniendo características del Tweet nº 78
Tweets del usuario: 620
[+] Retrieving last 200 tweets...


 89%|████████▉ | 178/200 [01:32<00:17,  1.27tw/s]